In [0]:
!pip install -U -q PyDrive

In [0]:
import sqlite3
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
!mkdir data

In [0]:
file_id = '1zQ8AR2Ry3ajzB3UZO1Sfk3xtDJlzQF2M'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('data/checking-logs.sqlite')

In [0]:
conn = sqlite3.connect('data/checking-logs.sqlite')

In [7]:
pd.io.sql.read_sql('pragma table_info(checker)', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,status,TEXT,0,None,0
2,2,success,INTEGER,0,None,0
3,3,timestamp,TIMESTAMP,0,None,0
4,4,numTrials,INTEGER,0,None,0
5,5,labname,TEXT,0,None,0
6,6,uid,TEXT,0,None,0


In [8]:
pd.io.sql.read_sql('select * from checker limit 10', conn)

,index,status,success,timestamp,numTrials,labname,uid
0,0,checking,0,2020-04-16 21:12:50.740474,5,None,admin_1
1,1,ready,0,2020-04-16 21:12:54.708365,5,code_rvw,admin_1
2,2,checking,0,2020-04-16 21:46:47.769088,7,None,admin_1
3,3,ready,0,2020-04-16 21:46:48.121217,7,lab02,admin_1
4,4,checking,0,2020-04-16 21:53:01.862637,6,code_rvw,admin_1
5,5,ready,0,2020-04-16 21:53:05.373389,6,code_rvw,admin_1
6,6,checking,0,2020-04-17 05:18:51.965864,1,None,None
7,7,ready,0,2020-04-17 05:19:02.744528,1,project1,user_4
8,8,checking,0,2020-04-17 05:22:35.249331,2,project1,user_4
9,9,ready,1,2020-04-17 05:22:45.549397,2,project1,user_4


In [0]:
checkers = pd.io.sql.read_sql(
    """
    select count(*)
    from pageviews
    where pageviews.uid in
        (
            select checker.uid
            from checker
            where status = 'ready' and numTrials = 1 and labname in ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
        )
    """,
    conn, columns=['cnt']
)

In [11]:
checkers

,count(*)
0,985


In [0]:
conn.close()